In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from scipy import stats
from tempfile import mkdtemp
from shutil import rmtree


from sklearn import set_config
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.ensemble import AdaBoostRegressor, VotingRegressor, GradientBoostingRegressor, StackingRegressor, RandomForestRegressor
from sklearn.feature_selection import SelectPercentile, mutual_info_regression, VarianceThreshold, SelectFromModel
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.metrics import make_scorer, mean_squared_error, mean_squared_log_error
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, OrdinalEncoder, StandardScaler, RobustScaler
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

In [2]:
data = pd.read_csv("../raw_data/X_y_data2.csv")
data = data[data.y < 2]
X = data.iloc[:, :-2]
y = data.iloc[:, -2:]

In [3]:
one_hot_category = ["state", "funding_status", "industry_groups", 'private_ipo', ]
ordinal_category = ["no_employees", "revenue_range"]
numerical_features = ['founded_year', 'private_ipo', 'website', 'phone', "no_founders",
       'email', 'linkedin', 'twitter', 'facebook', 'no_investors', 'no_fund_rounds', 'operting_status',
       'no_sub_orgs', 'has_preseed', 'has_seed', 'has_series_a', 'has_series_b', 'has_series_c',
       'has_series_d', 'has_series_e', 'has_angel', 'has_debt_financing',
       'has_grant', 'has_corporate_round', 'has_series_x']

In [4]:
no_employees_ordinal = [
    '11-50', '51-100', '101-250', '251-500', '501-1000', '1001-5000', '5001-10000', '10001+'
]

In [5]:
revenue_range_ordinal = [
    'Less than $1M', '$1M to $10M', '$10M to $50M', '$50M to $100M', '$100M to $500M', '$500M to $1B', '$1B to $10B', '$10B+'
]

In [6]:
feat_ordinal_dict = {
    "no_employees": no_employees_ordinal,
    "revenue_range": revenue_range_ordinal
}

In [7]:
encoder_ordinal = OrdinalEncoder(
    categories = [feat_ordinal_dict[i] for i in ordinal_category],  
    dtype = np.int64
)


preproc_ordinal = make_pipeline(
    SimpleImputer(strategy = "most_frequent"),
    encoder_ordinal, 
    MinMaxScaler()
)

preproc_ordinal

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('ordinalencoder',
                 OrdinalEncoder(categories=[['11-50', '51-100', '101-250',
                                             '251-500', '501-1000', '1001-5000',
                                             '5001-10000', '10001+'],
                                            ['Less than $1M', '$1M to $10M',
                                             '$10M to $50M', '$50M to $100M',
                                             '$100M to $500M', '$500M to $1B',
                                             '$1B to $10B', '$10B+']],
                                dtype=<class 'numpy.int64'>)),
                ('minmaxscaler', MinMaxScaler())])

In [8]:
preproc_min_numerical = make_pipeline(
    KNNImputer(),
    MinMaxScaler())

preproc_min_numerical

Pipeline(steps=[('knnimputer', KNNImputer()), ('minmaxscaler', MinMaxScaler())])

In [9]:
preproc_nominal = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    OneHotEncoder(handle_unknown="ignore")
)

preproc_nominal

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('onehotencoder', OneHotEncoder(handle_unknown='ignore'))])

In [10]:
preproc_robust_numerical = make_pipeline(
    KNNImputer(),
    RobustScaler())

preproc_robust_numerical

Pipeline(steps=[('knnimputer', KNNImputer()), ('robustscaler', RobustScaler())])

In [11]:
preproc = make_column_transformer(
        (preproc_ordinal, ordinal_category),
        (preproc_min_numerical, numerical_features),
        (preproc_nominal, one_hot_category),
        # (preproc_robust_numerical, robust_category),
        remainder="drop"
)

In [12]:
preproc

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(categories=[['11-50',
                                                                              '51-100',
                                                                              '101-250',
                                                                              '251-500',
                                                                              '501-1000',
                                                                              '1001-5000',
                                                                              '5001-10000',
                                                                              '10001+'],
                                                                             ['Less '
                                                                              'than '
                                                                              '$1M',
                                                                              '$1M '
                                                                              'to '
                                                                              '$10M',
                                                                              '$10M '
                                                                              'to '
                                                                              '$50M',
                                                                              '$50M '
                                                                              'to '
                                                                              '$100M',
                                                                              '$100M '
                                                                              'to '
                                                                              '$500M',
                                                                              '$500M '
                                                                              'to '
                                                                              '...
                                  'has_series_a', 'has_series_b',
                                  'has_series_c', 'has_series_d',
                                  'has_series_e', 'has_angel',
                                  'has_debt_financing', 'has_grant',
                                  'has_corporate_round', 'has_series_x']),
                                ('pipeline-3',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['state', 'funding_status', 'industry_groups',
                                  'private_ipo'])])

In [13]:
X_preprocessed = pd.DataFrame(preproc.fit_transform(X, y))

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y.iloc[:, -1:], test_size=.20, random_state=1)


In [15]:
! pip install tensorflow

In [16]:
from tensorflow import keras
from keras import Model, Sequential, layers, regularizers, optimizers
from keras.callbacks import EarlyStopping

2024-05-30 22:11:07.457970: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-30 22:11:07.461784: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-30 22:11:07.626507: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-30 22:11:08.126223: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-30 22:11:09.239920: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [17]:
X_preprocessed.shape

(855, 70)

In [18]:
reg = regularizers.l1_l2(l2=0.005)

model = Sequential()
model.add(layers.Input(shape=(X_preprocessed.shape[1],)))
model.add(layers.Dense(50, activation="relu", kernel_regularizer=reg))
model.add(layers.BatchNormalization(momentum=0.9))
model.add(layers.Dropout(rate=0.1))
model.add(layers.Dense(20, activation="tanh"))
model.add(layers.BatchNormalization(momentum=0.9))
model.add(layers.Dropout(rate=0.1))
model.add(layers.Dense(1, activation="sigmoid"))

2024-05-30 22:11:10.715877: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-30 22:11:10.716567: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [19]:
learning_rate = 0.01

optimizer = optimizers.Adam(learning_rate=learning_rate)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [20]:
es = EarlyStopping(
    patience=20,
    restore_best_weights=True,
    verbose=1
)

history = model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    epochs=700,
    batch_size=32,
    callbacks=[es],
    verbose=0
)

Epoch 75: early stopping
Restoring model weights from the end of the best epoch: 55.


In [21]:
history.history

{'accuracy': [0.541133463382721,
  0.5813528299331665,
  0.6581352949142456,
  0.7312614321708679,
  0.7751371264457703,
  0.7824497222900391,
  0.8208409547805786,
  0.8281535506248474,
  0.864716649055481,
  0.868372917175293,
  0.9012796878814697,
  0.8866544961929321,
  0.8921389579772949,
  0.8903107643127441,
  0.9085923433303833,
  0.9085923433303833,
  0.9323583245277405,
  0.9232175350189209,
  0.9195612668991089,
  0.9140768051147461,
  0.9287019968032837,
  0.9378427863121033,
  0.9378427863121033,
  0.9433272480964661,
  0.9414991140365601,
  0.9396709203720093,
  0.9305301904678345,
  0.9378427863121033,
  0.9360146522521973,
  0.9414991140365601,
  0.9542961716651917,
  0.9360146522521973,
  0.9378427863121033,
  0.9597806334495544,
  0.9561243057250977,
  0.9488117098808289,
  0.9488117098808289,
  0.9488117098808289,
  0.9652650952339172,
  0.9488117098808289,
  0.9597806334495544,
  0.97074955701828,
  0.9579524397850037,
  0.9670932292938232,
  0.972577691078186,
  0.

In [22]:
results = model.evaluate(X_test, y_test)


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8812 - loss: 0.4333 
